In [1]:
import re
import pandas as pd
import numpy as np
from collections import defaultdict
from nltk import tokenize


from nltk.sentiment.vader import SentimentIntensityAnalyzer

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('fivethirtyeight')
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from tqdm import tqdm_notebook as tqdm
from tqdm import trange


import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re


pd.set_option('display.max_rows', 500)


def getcorpus_local(local):
    with open(str(local)) as f:
        soup = BeautifulSoup(f, "html.parser", exclude_encodings=["iso-8859-7"])
        words = soup.get_text()
        #words= re.sub("[^a-zA-Z' ]+", " ", words)
        words = re.sub("\\n","",words)
        words =re.sub("\\'", "", words)
        return words
    
text = getcorpus_local(R"C:\Users\heather\Desktop\Post Kaiser Projects\The_room.TXT")



In [2]:
##############################################################################################################################
#
#
#                                           Import all the things!
#
#
##############################################################################################################################


import tensorflow as tf
import numpy as np
import os
import time

In [5]:
##############################################################################################################################
#
#
#                                           Get data
#
#
##############################################################################################################################


text[:1000]


#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


'THE ROOM, by Tommy P. Wiseau               ACT I               SCENE 1                YOU CAN SEE THE GOLDEN GATE BRIDGE, SUNRISE BEHIND THE BAY.                THEN AN EXTERNAL SHOT OF AN APARTMENT BUILDING SOUTH OF                MARKET STREET. THERE IS A SHOT OF A WINDOW OF THE ROOM. IT IS                FURNISHED SIMPLY.                AS WE PAN ACROSS THE ROOM WE SEE A MAN AND A WOMAN ASLEEP AND                PARTIALLY NAKED. THE ALARM CLOCK RINGS. THE MAN REACHES TO                THE CLOCK AND TURNS IT OFF. HE SLEEPILY AROUSES AND PUTS ON                HIS SHORTS AND WALKS SLOWLY TO THE BATHROOM. HE CLOSES THE                DOOR. PAN BACK TO THE WOMAN WAKING UP. THE MAN COMES OUT OF                THE BATHROOM AND SMILES TENDERLY AT HER.             THE ROOM by Tommy P. Wiseau Copyright Â©, 1999   Copyright Â©, 2001Â\xad15   Copyright Â©, 2000   ALL RIGHT RESERVED2.                              CONTINUED:                                             LISA               I am no

In [6]:
##############################################################################################################################
#
#
#                                           Process text
#
#
##############################################################################################################################




# Read, then decode for py2 compat.
#text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
#text =open(r"C:\Users\heather\Desktop\Post Kaiser Projects\Harry Potter\Harry Potter and the Sorcerer.txt", 'rb').read().strip()
#text = HPOne


# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 174407 characters


In [7]:
print(text[:250])

THE ROOM, by Tommy P. Wiseau               ACT I               SCENE 1                YOU CAN SEE THE GOLDEN GATE BRIDGE, SUNRISE BEHIND THE BAY.                THEN AN EXTERNAL SHOT OF AN APARTMENT BUILDING SOUTH OF                MARKET STREET. THE


In [8]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

80 unique characters


In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])


print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '"' :   2,
  '#' :   3,
  '$' :   4,
  '(' :   5,
  ')' :   6,
  ',' :   7,
  '.' :   8,
  '/' :   9,
  '0' :  10,
  '1' :  11,
  '2' :  12,
  '3' :  13,
  '4' :  14,
  '5' :  15,
  '6' :  16,
  '7' :  17,
  '8' :  18,
  '9' :  19,
  ...
}


In [10]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'THE ROOM, by ' ---- characters mapped to int ---- > [41 29 26  0 39 36 36 34  7  0 49 72  0]


In [11]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

T
H
E
 
R


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'THE ROOM, by Tommy P. Wiseau               ACT I               SCENE 1                YOU CAN SEE THE'
' GOLDEN GATE BRIDGE, SUNRISE BEHIND THE BAY.                THEN AN EXTERNAL SHOT OF AN APARTMENT BUI'
'LDING SOUTH OF                MARKET STREET. THERE IS A SHOT OF A WINDOW OF THE ROOM. IT IS          '
'      FURNISHED SIMPLY.                AS WE PAN ACROSS THE ROOM WE SEE A MAN AND A WOMAN ASLEEP AND '
'               PARTIALLY NAKED. THE ALARM CLOCK RINGS. THE MAN REACHES TO                THE CLOCK AN'


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'THE ROOM, by Tommy P. Wiseau               ACT I               SCENE 1                YOU CAN SEE TH'
Target data: 'HE ROOM, by Tommy P. Wiseau               ACT I               SCENE 1                YOU CAN SEE THE'


In [15]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 41 ('T')
  expected output: 29 ('H')
Step    1
  input: 29 ('H')
  expected output: 26 ('E')
Step    2
  input: 26 ('E')
  expected output: 0 (' ')
Step    3
  input: 0 (' ')
  expected output: 39 ('R')
Step    4
  input: 39 ('R')
  expected output: 36 ('O')


In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [19]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 80) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           20480     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 80)            82000     
Total params: 4,040,784
Trainable params: 4,040,784
Non-trainable params: 0
_________________________________________________________________


In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
sampled_indices

array([12, 61, 25, 68, 31, 13, 77, 32, 26,  8, 15, 30, 48, 31, 38, 11, 42,
       32, 31, 47,  4, 78, 76, 73, 69, 38, 35, 24,  3, 19, 44, 41, 60, 28,
       47, 13, 34, 51, 36, 67, 17, 49, 41, 17,  7, 11, 44, 54, 10, 29, 59,
       44, 71, 28,  5, 53, 53, 25,  0, 56, 17, 21, 77, 58, 50, 77, 36, 13,
       16, 45, 73, 18, 32, 19, 40, 11, 32, 71,  9, 55, 30, 10, 49, 47, 13,
       63, 29, 79, 58, 12, 30, 15, 67, 70,  0, 59, 44, 11, 67, 42],
      dtype=int64)

In [24]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '         CONTINUED: (4)                                             LISA               Ok, mom.     '

Next Char Predictions: 
 '2nDuJ3âKE.5IaJQ1UKJZ$€ÂzvQNC#9WTmGZ3MdOt7bT7,1Wg0HlWxG(ffD i7?âkcâO36Xz8K9S1Kx/hI0bZ3pH™k2I5tw lW1tU'


In [25]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 80)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.3883333


In [26]:
model.compile(optimizer='adam', loss=loss)

In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [28]:
EPOCHS = 100

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
 7/26 [=======>......................] - ETA: 38s - loss: 5.3309

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"JOHNNY "))